## Setup

In [23]:
#imports
import os, sys, json, datetime, re, xlrd  # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
import pandas as pd             # Provides data structures and data analysis tools
from openpyxl import Workbook
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import glob
import time

from tqdm import tqdm
from functools import reduce
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display_markdown
from cprl_functions.state_capture import thi_states,state_ref, state_coding, state_coding_r, state_pat, state_abv_pat
from cprl_functions.defined_functions import get_recent_file
# %%
os.chdir(r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files')

In [ ]:
#Get lookup Data
key_path = r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\connectors'
legislators_df = get_recent_file("*.csv", key_path)
# legislators_df = pd.read_csv(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\legislator data\connectors\leg_lookup_df.csv')

#grab scores data
dir_path = r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files'
activity_file = get_recent_file("activity*.csv", dir_path)
influence_file = get_recent_file("*infl*.csv", dir_path)

#make dfs
legislators_df = pd.read_csv(legislators_df)
activities_df = pd.read_csv(activity_file)
influence_df = pd.read_csv(influence_file)

#testing print statement
all_dfs = [legislators_df,activities_df,influence_df]
for all in all_dfs:
    print(*all.columns, sep = ',')

legislators_df

full_pk,primary_key,first_name,last_name
full_pk,primary_key,last_name,activities_score,events
full_pk,first_name,last_name,party,influence_score


,full_pk,primary_key,first_name,last_name
0,10100100,101001,Tim,Melson
1,10100200,101002,Tom,Butler
2,10100300,101003,Arthur,Orr
3,10100400,101004,Garlan,Gudger
4,10100500,101005,Greg,Reed
...,...,...,...,...
1988,57101500,571015,Charles,Trump
1989,57101601,571016,Jason,Barrett
1990,57101602,571016,Patricia,Rucker
1991,57101701,571017,Eric,Nelson


: 

In [ ]:
#narrow down scores dfs
# activities_df = activities_df.loc[:,['full_pk', 'activities_score']]
# influence_df = influence_df.loc[:,['full_pk', 'influence_score']]

: 

In [ ]:
#merging
#merge influence scores
first_merge = pd.merge(legislators_df, influence_df, how="left", on='full_pk', suffixes=('', '_y'))
first_merge.drop(first_merge.filter(regex='_y$').columns, axis=1, inplace=True)

# print(first_merge.to_string())
print(*first_merge.columns, sep=", ")
print(*activities_df.columns, sep=", ")



full_pk, primary_key, first_name, last_name, party, influence_score
full_pk, primary_key, last_name, activities_score, events


: 

In [ ]:

#merge influence scores
second_merge = pd.merge(first_merge, activities_df, how="left", on='full_pk', suffixes=('', '_y'))
second_merge.drop(second_merge.filter(regex='_y$').columns, axis=1, inplace=True)

second_merge

,full_pk,primary_key,first_name,last_name,party,influence_score,activities_score,events
0,10100100,101001,Tim,Melson,Republican,17.0,NaN,NaN
1,10100200,101002,Tom,Butler,Republican,11.0,NaN,NaN
2,10100300,101003,Arthur,Orr,Republican,18.0,NaN,NaN
3,10100400,101004,Garlan,Gudger,Republican,11.0,NaN,NaN
4,10100500,101005,Greg,Reed,Republican,20.0,NaN,NaN
...,...,...,...,...,...,...,...,...
1988,57101500,571015,Charles,Trump,Republican,12.0,NaN,NaN
1989,57101601,571016,Jason,Barrett,Republican,12.0,NaN,NaN
1990,57101602,571016,Patricia,Rucker,Republican,11.0,NaN,NaN
1991,57101701,571017,Eric,Nelson,Republican,12.0,NaN,NaN


: 

In [30]:

# Function to count non-whitespace events
def count_events(event_str):
    if pd.isna(event_str):  # Check if the value is NaN
        return 0
    # Split by "|" and strip whitespace
    events_list = [event.strip() for event in event_str.split('|') if event.strip()]
    
    return len(events_list)

# Add a new column with the count of events
second_merge['event_count'] = second_merge['events'].apply(count_events)
# print(second_merge.columns)
# print(second_merge.to_string())



In [31]:
from datetime import date

#final exports

final_df = second_merge.copy()
print(*final_df.columns, sep=', ')


full_pk, primary_key, first_name, last_name, party, influence_score, activities_score, events, event_count


In [36]:

for i,j in enumerate(final_df['full_pk']):
    j_pk = re.findall(r'^\d{6}', str(j))[0]
    seat_num = re.findall(r'\d{2}$', str(j))[0]
    # print(f'seat num is {seat_num}')
    # print(f'type is {type(seat_num)}')
    
    if seat_num == '00':
        # print('its a single seat')
        seat_num_v = np.nan
    else:
        seat_num_v = 'Seat ' + seat_num

    
    state_match = re.findall(r'^\d{2}', str(j_pk))
    state = state_coding_r.get(int(state_match[0]))
    chamber = int(re.findall(r'(?<=^\d{2})\d{1}(?=\d{3})', str(j_pk))[0])
    
    if chamber == 0:
        chamber_v = 'House'
    else:
        chamber_v = 'Senate'
    
    district = int(re.findall(r'(?<=^\d{3})\d{3}$', str(j_pk))[0].lstrip('0'))
    district_v = f'District {district}'
    
    
    
    if str(seat_num_v) != 'nan':
        final_df.loc[i,'seat_num'] = seat_num_v
        


    # print('################')
    # print(f'state is {state}')
    # print(f'chamber is {chamber_v}')
    # print(f'district is {district_v}')
    # print(seat_num_v)
    

    final_df.loc[i,'state'] = state
    final_df.loc[i,'chamber'] = chamber_v
    final_df.loc[i,'district'] = district_v

print(*final_df.columns, sep=', ')


final_df.loc[:,['year']] = "2024"
   

full_pk, primary_key, first_name, last_name, party, influence_score, activities_score, events, event_count, state, chamber, district, seat_num


In [43]:

file_name = f'compiled_scores{str(date.today()).replace('-','_')}.xlsx'
csv_file_name = f'compiled_scores{str(date.today()).replace('-','_')}.csv'
final_df.to_excel(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\{file_name}', index=False)
final_df.to_csv(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\{csv_file_name}', index=False)
final_df

,full_pk,primary_key,first_name,last_name,party,influence_score,activities_score,events,event_count,state,chamber,district,seat_num,year
0,10100100,101001,Tim,Melson,Republican,17.0,NaN,NaN,0,AL,Senate,District 1,NaN,2024
1,10100200,101002,Tom,Butler,Republican,11.0,NaN,NaN,0,AL,Senate,District 2,NaN,2024
2,10100300,101003,Arthur,Orr,Republican,18.0,NaN,NaN,0,AL,Senate,District 3,NaN,2024
3,10100400,101004,Garlan,Gudger,Republican,11.0,NaN,NaN,0,AL,Senate,District 4,NaN,2024
4,10100500,101005,Greg,Reed,Republican,20.0,NaN,NaN,0,AL,Senate,District 5,NaN,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1988,57101500,571015,Charles,Trump,Republican,12.0,NaN,NaN,0,WV,Senate,District 15,NaN,2024
1989,57101601,571016,Jason,Barrett,Republican,12.0,NaN,NaN,0,WV,Senate,District 16,Seat 01,2024
1990,57101602,571016,Patricia,Rucker,Republican,11.0,NaN,NaN,0,WV,Senate,District 16,Seat 02,2024
1991,57101701,571017,Eric,Nelson,Republican,12.0,NaN,NaN,0,WV,Senate,District 17,Seat 01,2024


pulling in bills

In [40]:

bills_file = r"C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\bills\leg_bills_info.xlsx"

bills = pd.read_excel(bills_file)
print(bills.columns)


Index(['primary_key', 'bill_labels', 'bill_counts'], dtype='object')


In [ ]:

compiled_plus_bills = pd.merge(final_df, bills, how="left", on='primary_key')
compiled_plus_bills


In [44]:

file_name_cb = f'compiled_plus_bills{str(date.today()).replace('-','_')}.xlsx'
csv_file_name_cb = f'compiled_plus_bills{str(date.today()).replace('-','_')}.csv'
compiled_plus_bills.to_excel(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\{file_name_cb}', index=False)
compiled_plus_bills.to_csv(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\{csv_file_name_cb}', index=False)


In [ ]:
bills_and_legislators = compiled_plus_bills.assign(
    bills=compiled_plus_bills['bill_labels'].str.split('|')  # Split the string into a list
).explode('bills')  # Create a new row for each list element

# Optionally, clean up the brackets
# bills_and_legislators['Values'] = df_expanded['Values'].str.strip('[]')
# print(compiled_plus_bills.columns)
bills_and_legislators

In [50]:

file_name_bl = f'bills_and_legislators{str(date.today()).replace('-','_')}.xlsx'
csv_file_name_bl = f'bills_and_legislators{str(date.today()).replace('-','_')}.csv'
bills_and_legislators.to_excel(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\{file_name_bl}', index=False)
bills_and_legislators.to_csv(fr'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\Legislators Data\leg_data_update_10_2024\build files\compiled scores\{csv_file_name_bl}', index=False)


In [ ]:

# , suffixes=('', '_y'))
# second_merge.drop(second_merge.filter(regex='_y$').columns, axis=1, inplace=True)

In [ ]:



# print(final_df.columns)

# for i,j in enumerate(final_df['full_pk']):
#     print('#############')
#     print(j)




# final_df